In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import pandas as pd
import numpy as np
import random

In [40]:
df = pd.read_csv('/content/drive/MyDrive/Minor_Project/sem7/dataset.csv')
algorithms = df['Algorithm'].unique()
print(algorithms)

['Binary Search Algorithm' 'Bubble Sort Algorithm'
 'Depth-First Search (DFS) Algorithm' "Dijkstra's Shortest Path Algorithm"
 'Factorial (Recursive)' 'Fibonacci (Iterative)' 'GCD Algorithm'
 'Merge Sort Algorithm' 'Quick Sort Algorithm' 'Selection Sort Algorithm']


In [41]:
def fitness(chromosome):
    # Test cases
    total_fail = 0
    total_pass = 0
    total = 0
    for idx in range(len(chromosome)):
        if chromosome[idx] == 1:
            # Get data for the current algorithm, handling potential errors
            try:
                # Use .iloc[0] to get the first value if multiple entries exist
                # If the algorithm is not found, it will raise an IndexError which is handled
                # by the except block
                failed_tests = df.loc[df['Algorithm'] == algorithms[idx]]['Fails'].iloc[0]
                passed_tests = df.loc[df['Algorithm'] == algorithms[idx]]['Passes'].iloc[0]
                total_tests = df.loc[df['Algorithm'] == algorithms[idx]]['Total'].iloc[0]

                total_fail += failed_tests
                total_pass += passed_tests
                total += total_tests
            except (IndexError, KeyError):
                # If the algorithm is not found or the column is missing, skip it
                # print(f"Algorithm {algorithms[idx]} not found in DataFrame or missing columns.")  # Optional debug message
                pass  # Or you could assign a default value like total_fail += 0

    try:
        result = (((0.8 * total_fail) + (0.2 * total_pass)) / total)
    except ZeroDivisionError:
        result = float('inf')  # or any other appropriate value when total is 0

    return result

In [42]:


def select(population):
    # Randomly select two chromosomes and pick the higher fitness score
    selected = random.sample(population, 2)
    return max(selected, key=lambda chromo: chromo['fitness'])


In [43]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 2)  # Choose a point to swap
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


In [44]:
def mutate(chromosome):
    mutation_point = random.randint(0, 2)
    chromosome[mutation_point] = 1 - chromosome[mutation_point]  # Flip the bit


In [45]:
# GA Parameters
num_generations = 50
mutation_rate = 0.1
# Initialize population
population_size = 10
population = [{'chromosome': [random.randint(0, 1) for _ in range(len(algorithms))], 'fitness': 0} for _ in range(population_size)]

# Calculate fitness for the initial population
for individual in population:
    individual['fitness'] = fitness(individual['chromosome'])



# Main GA loop
for generation in range(num_generations):
    # Sort population by fitness (higher is better)
    population = sorted(population, key=lambda x: x['fitness'])

    # Create a new population
    new_population = []
    while len(new_population) < population_size:
        # Select parents
        parent1 = select(population)['chromosome']
        parent2 = select(population)['chromosome']

        # Perform crossover
        child1, child2 = crossover(parent1, parent2)

        # Perform mutation
        if random.random() < mutation_rate:
            mutate(child1)
        if random.random() < mutation_rate:
            mutate(child2)

        # Add children to the new population
        new_population.append({'chromosome': child1, 'fitness': fitness(child1)})
        new_population.append({'chromosome': child2, 'fitness': fitness(child2)})

    # Replace the old population with the new population
    population = new_population

# Output final solution
result=population[0]['chromosome']

for index,val in enumerate(result):
   if val == 1:
      print("function",algorithms[index],"is faulty")


function Depth-First Search (DFS) Algorithm is faulty
function Dijkstra's Shortest Path Algorithm is faulty
function Merge Sort Algorithm is faulty
function Selection Sort Algorithm is faulty
